In [ ]:
#| default_exp toolbox

# Methods in Evolutionary Game Theory

> A set of methods for solving Evolutionary Games (see Nowak 2006 and the references section)

## Utilities

In [ ]:
def thread_macro(current_value, *funcs, identifier="self"):
    """Pipes current_value through each function in funcs.

    Each element in funcs is either a function or a list/tuple containing
    a function followed by its other arguments.
    This function imitates the Clojure as-if threading macro.

    Notes: By default current_value is threaded as the first argument of the
    function call. Yet, one can use the syntax [func, arg1, "self", arg2] (or
    (func, arg1, "self", arg2)) so that current_value will instead be threaded
    in whatever place "self" would be. If you need to, you can set this "self"
    identifier to a different value.
    """

    for func in funcs:
        if isinstance(func, (list, tuple)):
            place = 0
            for i, el in enumerate(func[1:]):
                if el == identifier:
                    place = i
                    func = [el for el in func if el != identifier]
            func, args1, args2 = func[0], func[1:place + 1], func[place + 1:]
            current_value = func(*args1, current_value, *args2)
        else:
            current_value = func(current_value)
    return current_value

In [ ]:
def broadcast_concatenate_axes(ax1, ax2):
    """Broadcast both numpy axes and concatenate along last dimension"""
    ax1new = ax1
    for _ in range(np.ndim(ax2) - 1):
        ax1new = ax1new[..., None, :]
    ax2new = ax2
    for _ in range(np.ndim(ax1) - 1):
        ax2new = ax2new[None, ..., :]
    ax1new = np.broadcast_to(ax1new,
                             (*ax1.shape[:-1], *ax2.shape[:-1], ax1.shape[-1]))
    ax2new = np.broadcast_to(ax2new,
                             (*ax1.shape[:-1], *ax2.shape[:-1], ax2.shape[-1]))
    ax = np.concatenate((ax1new, ax2new), axis=-1)
    return ax

In [ ]:
def build_parameter_grid_from_axes(axes):
    """Build a numpy array with all combinations of elements specified in axes.

    Each axis in axes gives an array of values that should be repeated for each
    value in the other axes. Primitive types and lists of primitive types are
    first promoted to numpy arrays.
    """

    dtypes = (float, int, bool, str)
    for i, axis in enumerate(axes):
        condition = isinstance(axis, dtypes) or all(
            isinstance(el, dtypes) for el in list(axis))
        axes[i] = np.array([axis]).T if condition else axis
    tensor = functools.reduce(broadcast_concatenate_axes, axes)
    return tensor.reshape((-1, tensor.shape[-1]))

## Finite Markov Chains

We examine a finite population of players using different strategies who engage in social learning.

In the limit of small mutations, most of the time everyone plays the same strategy. States in which everyone plays the same strategy are known as **monomorphic states**. Occassionally, mutant strategies can fixate in the population, resulting in everyone adopting the same new strategy. We can use Markov Chains to analyse the relative frequencies with which each strategy is played by the population.

The steps for computing the ergodic (i.e. long-run) strategy distribution is as follows:

1. Build a transition matrix between all monomorphic states
2. Find the unit eigenvector associated with this transition matrix

In [ ]:
def build_transition_matrix(models):
    """Build a transition matrix between all monomorphic states
    using the fermi social learning rule for each model."""
    
    Z, S, W, β = [models[k] for k in ['Z','strategy_set', 'W', 'β']]
    π = models['payoffs']
    M = np.zeros(( W.shape[0], len(S), len(S)))
    for row_ind, s in enumerate(S):
        for col_ind, sₒ in enumerate(S):
            if row_ind == col_ind:
                continue
            πAA = π[:, row_ind, row_ind][:, None, None]
            πAB = π[:, row_ind, col_ind][:, None, None]
            πBA = π[:, col_ind, row_ind][:, None, None]
            πBB = π[:, col_ind, col_ind][:, None, None]
            ΠA = [πAA*(k-1)/Z + πAB*(Z-k)/Z
                  for k in range(Z+1)]
            ΠB = [πBA*k/Z + πBB*(Z-k-1)/Z
                  for k in range(Z+1)]
            Tplus = [(Z - k)/Z
                     * k/Z
                     * (1 + np.exp(-β*(ΠA[k] - ΠB[k])))**-1 
                     for k in range(Z+1)]
            Tneg = [(Z - k)/Z
                    * k/Z
                    * (1 + np.exp(β*(ΠA[k] - ΠB[k])))**-1
                    for k in range(Z+1)]
            ρ = (1 
                 + np.sum([np.prod([Tneg[j]/Tplus[j]
                                    for j in range(1,i+1)],
                                   axis=0)
                           for i in range(1,Z)], axis=0))**-1
            M[:, col_ind, row_ind] = ρ[:,0,0]/(len(S)-1)
    for row_ind in range(len(S)):
        col_inds = [i for i in range(len(S)) if i != row_ind]
        no_move = 1 - np.sum(M[:, row_ind, col_inds], axis=1)
        M[:, row_ind, row_ind] = no_move
    return {**models, "transition_matrix": M}

In [ ]:
def find_ergodic_distribution(models):
    """Find the ergodic distribution of a markov chain with the
    given transition matrix."""
    
    M = models.get("transition_matrix", np.zeros((1, 1, 1)))
    # find unit eigenvector of markov chain
    Λ,V = np.linalg.eig(M.transpose(0,2,1))
    x = np.isclose(Λ, 1)
    # if multiple unit eigenvalues then choose the first
    y = np.zeros_like(x, dtype=bool)
    idx = np.arange(len(x)), x.argmax(axis=1)
    y[idx] = x[idx]
    ergodic = np.array(V.transpose(0,2,1)[y], dtype=float)
    # ensure ergodic frequencies are positive and sum to 1
    ergodic = np.abs(ergodic) / np.sum(np.abs(ergodic), axis=1)[:, None]
    return {**models, 'ergodic':ergodic}

In [ ]:
def markov_chain(models):
    """Find the ergodic distribution of the evolutionary
    game given by each model in models."""
    return thread_macro(models,
                        build_transition_matrix,
                        find_ergodic_distribution)

## TODO

- add unit tests for markov_chain
- add integration tests for makov_chain
- Split build_transition_matrix into more components and provide tests and documentation for each
  - a fermi learning function
  - a fixation rate equation
- Move utilities to its own notebook or to its own package

[Tips on using Latex in Markdown](https://towardsdatascience.com/write-markdown-latex-in-the-jupyter-notebook-10985edb91fd)

### Fixation rate

The fixation rate for type B in a population of type A is defined as the probability that the appearance of a mutant of type B leads to the entire population adopting type B instead of A, i.e. what is the likelihood that a mutant of type A invades population B.

A derivation of the fixation rate defined above can be found in Nowak 2006 (reproduced below).

> Consider a one-dimensional stochastic process on a discrete state space, $ i \in \{0, 1, \cdots, N\}$ that represents the number of individuals in a population of $N$ individuals who are of type $A$, the rest are type $B$.
>
> In each stochastic event, the number of individuals of type $A$ can at most increase or decrease by 1.
>
> For a given number of individuals, $i$, let $a_i$, $b_i$, and $1 - a_i - b_i$ represent the chance of an increase, decrease, or no change in $i$.
> 
> This stochastic process follows the transition matrix ,$P$ (*not to be confused with the transition matrices we discuss elsewhere!*)
>
>
> \begin{equation}
P \, = \, \begin{pmatrix}
1 & 0 & 0 & \cdots & 0 & 0 & 0\\
b_1 & (1 - a_1 - b_1) & a_1 & \cdots & 0 & 0 & 0\\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & b_{n-1} & (1 - a_{n-1} - b_{n-1}) & a_{n-1}\\
0 & 0 & 0 & \cdots & 0 & 0 & 1\\
\end{pmatrix}
\end{equation}
>
> Denote by $x_i$ the probability of reaching state $N$ when starting from $i$.
>
> From transition matrix $P$ above, we can see that $x_i$ must satisfy:
>
> $ x_0 = 0 $
>
> $ x_i = b_i x_{i-1} + (1 - a_i - b_i) x_i + a_i x_{i+1}$
>
> $ x_N = 1 $
>
> The fixation rate for a mutant A in a population of type B is clearly $x_1$
>
> We can solve for $x_i$ by rewriting the above as $ b_i x_i - b_i  x_{i-1} = a_i x_{i+1} - a_i x_i $.
> 
> We can denote $y_i = x_i - x_{i-1}$ to simplify the above to $ y_{i+1} = \frac{b_i}{a_i} y_i$
>
> Notice that $\sum_{i=1}^N{y_i} = x_N - x_0 = 1$ and that $y_1 = x_1$
>
> We can use the above to write
\begin{equation}
x_1 + {\sum_{i=2}^N{y_i}} = x_1 (1 + {\sum_{i=1}^{N-1}{\prod_{j=1}^{i} \frac{b_j}{a_j}}}) = 1
\end{equation}
>
> And so
\begin{equation}
x_1 = \frac{1}{(1 + \sum_{i=1}^{N-1}{\prod_{j=1}^{i} \frac{b_j}{a_j}})}
\end{equation}
>
> Note that $x_1$ is the fixation rate for a mutant $A$ in a population of type $B$
>
> *Also note that $x_{N-1}$ is the fixation rate for a mutant $B$ in a population of type $A$. We could find expressions for all $x_i$ if we note that $x_i = x_1 (1 + \sum_{j=1}^{i-1}{\prod_{k=1}^{j} \frac{b_k}{a_k}})$ (see Nowak 2006 for further details).*

In [ ]:
ΠA = [πAA*(k-1)/Z + πAB*(Z-k)/Z
                  for k in range(Z+1)]
            ΠB = [πBA*k/Z + πBB*(Z-k-1)/Z
                  for k in range(Z+1)]
            Tplus = [(Z - k)/Z
                     * k/Z
                     * (1 + np.exp(-β*(ΠA[k] - ΠB[k])))**-1 
                     for k in range(Z+1)]
            Tneg = [(Z - k)/Z
                    * k/Z
                    * (1 + np.exp(β*(ΠA[k] - ΠB[k])))**-1
                    for k in range(Z+1)]
            ρ = (1 
                 + np.sum([np.prod([Tneg[j]/Tplus[j]
                                    for j in range(1,i+1)],
                                   axis=0)
                           for i in range(1,Z)], axis=0))**-1

IndentationError: unexpected indent (946315720.py, line 3)